In [27]:
import csv
import itertools
import re
import pandas as pd
# import astral


#* ENVS
wspr_csv = "2022-02-05-flt11.csv"
time_slot = 4
re_time_slot = f'^..:.{time_slot}'
parsed_csv = f'parsed_{wspr_csv}'

header = []
parsed_dt = []
alt = []


def alt_calc(power,split_ts):
    if split_ts == None:
        alt_m = int(power)*20 #* Type 3 WSPR message decode x 20 (20m per dBm)
    else:
        alt_m = int(power)*300 #* Type 1 or 2 WSPR message decode x 300 (300m per dBm)
    alt_ft = alt_m * 3.28 #* Convert meters to feet
    return (alt_m, int(alt_ft))


with open(wspr_csv, 'r') as file:
    data = csv.reader(file)
    header = next(data)
    for row in data:
        alt = []
        split_dt = row[1].split(' ')
        split_ts = re.search(re_time_slot, split_dt[1])
        alt_m = alt_calc(row[15], split_ts)
        alt.extend(alt_m)
        parsed_dt.append(list(itertools.chain(row, split_dt, alt)))

add_headers = ['date', 'time', 'altitude_m', 'altitude_ft']
header.extend(add_headers)

with open(parsed_csv, "w",newline="") as f:
    writer = csv.writer(f)
    header[1] = 'datetime'
    # print(header)
    writer.writerow(header)
    for row in parsed_dt:
        writer.writerow(row)

In [28]:
#* Adding in a dataframe to work to add the WSPR 3 packet to the Type 1 and 2

parsed_df = pd.read_csv(parsed_csv, parse_dates=["datetime"], usecols=['datetime','date','time', 'tx_lat', 'tx_lon', 'tx_loc', 'altitude_m', 'altitude_ft'])
parsed_df.tail()

,datetime,tx_lat,tx_lon,tx_loc,date,time,altitude_m,altitude_ft
3708,2022-02-04 03:34:00,37.021,144.708,QM27ia,2022-02-04,03:34:00,8100,26568
3709,2022-02-04 03:36:00,37.104,145.542,QM27sc,2022-02-04,03:36:00,260,852
3710,2022-02-04 03:36:00,37.104,145.542,QM27sc,2022-02-04,03:36:00,260,852
3711,2022-02-04 03:36:00,37.104,145.542,QM27sc,2022-02-04,03:36:00,260,852
3712,2022-02-04 03:36:00,37.104,145.542,QM27sc,2022-02-04,03:36:00,260,852


In [29]:
parsed_df.dtypes

datetime       datetime64[ns]
tx_lat                float64
tx_lon                float64
tx_loc                 object
date                   object
time                   object
altitude_m              int64
altitude_ft             int64
dtype: object

In [30]:
parsed_df['type'] = parsed_df['time'].str[4]

In [31]:
# parsed_df.drop_duplicates(subset ="datetime", keep = False, inplace = True)
# parsed_df.shape

In [32]:
parsed_df

,datetime,tx_lat,tx_lon,tx_loc,date,time,altitude_m,altitude_ft,type
0,2022-01-26 17:34:00,42.188,-87.792,EN62ce,2022-01-26,17:34:00,0,0,4
1,2022-01-26 17:34:00,42.188,-87.792,EN62ce,2022-01-26,17:34:00,0,0,4
2,2022-01-26 17:34:00,42.188,-87.792,EN62ce,2022-01-26,17:34:00,0,0,4
3,2022-01-26 17:34:00,42.188,-87.792,EN62ce,2022-01-26,17:34:00,0,0,4
4,2022-01-26 17:34:00,42.188,-87.792,EN62ce,2022-01-26,17:34:00,0,0,4
...,...,...,...,...,...,...,...,...,...
3708,2022-02-04 03:34:00,37.021,144.708,QM27ia,2022-02-04,03:34:00,8100,26568,4
3709,2022-02-04 03:36:00,37.104,145.542,QM27sc,2022-02-04,03:36:00,260,852,6
3710,2022-02-04 03:36:00,37.104,145.542,QM27sc,2022-02-04,03:36:00,260,852,6
3711,2022-02-04 03:36:00,37.104,145.542,QM27sc,2022-02-04,03:36:00,260,852,6


In [33]:
parsed_df.to_csv('new_wspr.csv', index = False)

In [34]:
# I only print for type 0 packets and add the 0 detail to type 8 base.
with open("new_wspr.csv", "r") as a_file:
    for line in a_file:
        spline = line.split(',')
        # print(spline[8].strip('\n'))
        if spline[8].strip('\n') == '4':
            # print("it's an 8")
            base_altitude = int(spline[7])
        elif spline[8].strip('\n') == '6':
            altitude = base_altitude + int(spline[7])
            print(f"{spline[4]} {spline[5]} {altitude} ft")

2022-01-26 17:36:00 1115 ft
2022-01-26 17:36:00 1115 ft
2022-01-26 17:36:00 1115 ft
2022-01-26 17:36:00 1115 ft
2022-01-26 17:46:00 1967 ft
2022-01-26 17:46:00 1967 ft
2022-01-26 17:46:00 1967 ft
2022-01-26 17:46:00 1967 ft
2022-01-26 17:46:00 1967 ft
2022-01-26 17:46:00 1967 ft
2022-01-26 17:46:00 1967 ft
2022-01-26 17:46:00 1967 ft
2022-01-26 17:46:00 1967 ft
2022-01-26 17:46:00 1967 ft
2022-01-26 17:46:00 1967 ft
2022-01-26 17:46:00 1967 ft
2022-01-26 17:46:00 1967 ft
2022-01-26 17:46:00 1967 ft
2022-01-26 17:46:00 1967 ft
2022-01-26 17:46:00 1967 ft
2022-01-26 17:46:00 1967 ft
2022-01-26 17:46:00 1967 ft
2022-01-26 17:46:00 1967 ft
2022-01-26 17:46:00 1967 ft
2022-01-26 17:46:00 1967 ft
2022-01-26 18:06:00 3804 ft
2022-01-26 18:06:00 3804 ft
2022-01-26 18:06:00 3804 ft
2022-01-26 18:06:00 3804 ft
2022-01-26 18:06:00 3804 ft
2022-01-26 18:16:00 4919 ft
2022-01-26 18:16:00 4919 ft
2022-01-26 18:16:00 4919 ft
2022-01-26 18:16:00 4919 ft
2022-01-26 18:16:00 4919 ft
2022-01-26 18:16:00 

Now want to take the altitude in the :08 mins and add to the following :00 mins. to get teh precise altitude.

```
date,time,altitude_m,altitude_ft
2022-01-07,23:38:00,6900,22632
2022-01-07,23:40:00,1060,3476
```

For example the precise altitude is `7960 m`